<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/Question_Generation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install simpletransformers
from simpletransformers.t5 import T5Model
#https://towardsdatascience.com/asking-the-right-questions-training-a-t5-transformer-model-on-a-new-task-691ebba2d72c
!pip install convokit
import pandas as pd
from convokit import Corpus, download
corpus = Corpus(filename=download("parliament-corpus"))
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 194kB 2.7MB/s 
     |████████████████████████████████| 675kB 7.5MB/s 
     |████████████████████████████████| 204kB 12.4MB/s 
     |████████████████████████████████| 3.8MB 15.7MB/s 
     |████████████████████████████████| 890kB 34.9MB/s 
     |████████████████████████████████| 1.1MB 42.9MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=84efb1a67f989b73b39da6844f4813ac6a91ec4521a62501cb4f5c49c4e2791b
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9db895ababa06c5e3f6950eb8a444079069a8e58f2df386d70e72c6e41531858
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses
     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 1.4MB

In [3]:
rows_list = []
for convo in corpus.iter_conversations():
  utts=convo.get_chronological_utterance_list()
  dict1 = {}
  if len(utts)==2:
    dict1['prefix']='question-generation'
    dict1['input_text']=utts[1].text
    dict1['target_text']=utts[0].text 
    rows_list.append(dict1)
df = pd.DataFrame(rows_list)
train_df,test_df=train_test_split(df, shuffle=True, random_state=36)

In [4]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False,
}

model = T5Model("t5-base", args=model_args)

model.train_model(train_df, eval_data=test_df)

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


Running loss: 4.289723

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 2.556780

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 2.653446


In [5]:
!cp -r outputs/best_model 'drive/My Drive/answers'

In [6]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "eval_batch_size": 128,
    "num_train_epochs": 1,
    "save_eval_checkpoints": False,
    "use_multiprocessing": False,
    "num_beams": None,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

model = T5Model('drive/My Drive/answers/best_model', args=model_args)
df1=df.iloc[1:5]

preds = model.predict(
    ["question-generation: " + description for description in df1["input_text"].tolist()]
)

In [7]:
for speech in range(len(preds)):
  print('Context')
  print(df.iloc[speech+1]["input_text"])
  print('Truth')
  print(df.iloc[speech+1]["target_text"])
  print('Generated')
  for generated in preds[speech]:
    print(generated)
  print()

Context
I assure my hon Friend that the Government are fully signed up to what he describes as the Patten vision , and that we will continue to deliver on it . That will include making sure that the PSNI reflects more fully the composition of Northern Ireland 's population . Over the past three years , we have committed £ 178 million to fulfilling the Patten vision for policing in Northern Ireland . He has told the House about the events in Derry , and I of course join him and others in condemning dissident republicans who continue to think that violence and conflict are the way forward , when the truth is that politics is the way forward . Equally , however , I condemn those loyalist paramilitaries who were involved in the shooting of a police officer at the weekend . That is just as damnable , and such actions are increasingly marginal in a Northern Ireland society where real politics is taking over and determining the future .
Truth
Will the Minister assure the House that budget con

In [8]:
american_speeches=['Mr. Speaker, I thank the gentleman. Mr. Speaker, you dont have to look far to see that families across this country are gripped with a tremendous amount of fear and uncertainty. They fear for their jobs, if they have one. They fear for their future as they see their 401(k)s, their college savings accounts collapse. They fear that their elected leaders dont get it. They fear that this Congress may very well be incapable of change, incapable of producing the kind of results that they want and to get it right. Under existing House rules, when a bill is brought to the floor that includes a tax increase, the minority has a right to offer a motion to strike that increase, and the Republican minority had done that on nearly half a dozen occasions over the past 2 years. With this rule change now, though. House Democrats are trying to push through what we Republicans will no longer have, the ability to say "no" to higher taxes. We will not be able to simply strike a tax increase and demand an up or down vote. In fact, the only option we will have would be to replace one tax increase with another. There will be no ability for us to cut taxes to lighten the burden on the middleclass families that are hurting right now. One can see that this rule change makes it a lot easier for the Democrat majority to In fact hide tax increases inside other larger bills. In fact, that is why all of us are sitting here scratching our heads. If the House Democrats feel a tax increase is necessary, then why wouldnt they allow for a full and open debate? Why not let the American people have a say? Why not let the hardworking people of this country hear why Washington is once again looking to take more of their hard earned money? Either way, what is clear, this type of partisan rules change flies in the face of a new era of openness and transparency that Presidentelect Obama has promised, I take the Presidentelect at his word, I believe he wants transparency, openness, and debate, I believe he wants Washington to begin to do business differently, I believe he is serious in wanting Congress to work together for the good of all of our constituents. But apparently that word hasnt made its way down to the leadership of the House.',
                   'I thank the gentleman very much. Mr. Speaker, I am beginning my 31st year here, and one of the things that I have learned both being in the majority and being in the minority is that procedural fairness is the antithesis to partisanship, I want to repeat that: Procedural fairness is the antithesis to partisanship. This rules package, and particularly the changes in the motion to recommit, will bring about more partisanship, and I would ask my friends on the majority side to reconsider what they are proposing here. The previous speakers on the Republican side have stated instances in the last 2 years where it has resulted in excessive partisanship because of changes that have been made to the motions to recommit on an ad hoc basis allowing the majority to pull the bill, their choice, not ours, because they set the schedule, not having motions to recommit on certain bills and not allowing to strike proposed tax Increases. What is wrong with debating these issues? And what is wrong if the majority of this House of Representatives, which is 21 seats more Democratic than the one that just expired, agrees with the Republican minority every once in awhile? What are you afraid of? Are you afraid of losing a few more motions to recommit? If that is the motivation behind this, shame on you, because you are shutting down the process and you are going to result in more partisanship, not less. You are going to result in having the country even more divided, not less, and that goes exactly against what our new President has been trying to do with practically everything he said since he won the election 2 months ago.','The gentleman from Wisconsin said, why will the majority not in some instances agree with the minority? Thats the problem. We are talking about cases where we in the majority have tried to agree with the minority, and they would not be agreed with. They would not take yes for an answer. This is the issue: if they offer a motion to recommit and it says forthwith, and they win the vote, the bill is amended. If they offer an amendment to a bill, not having offered it in committee, not having gone to the Rules Committee to ask it to be on the floor, if they take a noncontroversial popular issue and offer it as the motion to recommit, but say it should be sent to the committee and reported back promptly, we have tried to agree with them, and they have refused. This literally is a way to not take yes for an answer, its a way to take something to which the majority would like to agree, I have been here when I, and when the majority leader has said, in such a situation, could we get unanimous consent to simply agree to that now, and the minority has said no. Well, people have a right not to be agreed with. People have a right not to be agreeable. Some indulge that right more than others. But you dont have a right to refuse to be agreed with, and then complain that you werent agreed with. And thats all thats at stake here. So, yes, there are times when the majority should say yes to the minority, and that should be determined by the floor. What were saying is the minority should not manufacture a situation in which there is no way to say yes to them because their goal is patently not to amend that particular bill, because if it was, they would accept the request that that amendment be accepted. Instead, it is to put a bill back to committee because theyre afraid to vote against it. Thats the issue. This is used as a way to send bills back to committee to avoid votes. And this leaves, this package, the minority, fully able to offer any motion to recommit or send it back to committee. It just says they cant play games.',
                   'Mr. Presiden with great admiration and resp I take this time to recognize Kentuckys most distinguish zens. Roman Catholic Bisho McRaith, who retired as th Bishop of the Diocese of Owensb Bishop McRaiths service o last 26 years in the Dio Owensborowhich consists of ties with 79 parishes, 3 high sc middle schools and 13 ele schoolshas made him a legac community. In addition to being a large Owensboro Diocese is one of t diverse dioceses--home to a larl ber of Hispanic Catholic imm along with a priesthood that men from Latin America. Asia, rica. The work done by Bishop and the priests at Owensboro has increased church attendanc E. Vol, 155. Pt, 1 January 6, 2009 e need to els that are considered among the highrun this est in the Nation, rs man, I Bishop McRaith has left his commuy and ac- nity a better place because of the auave faith Move for- thenticity and kindness of his services e to ruin and faith. While I am sad to see him retire, I am comforted knowing that those who learned from him will conDAVID, tinue the good work that he displayed each day. On behalf of all of those who of Idaho tor] held uld do to thank Bishop John McRaith for the better in- grace and strength he brought to westy and the em Kentucky., s we are uture, we ract. The TRIBUTE TO HARRIET CORNELL e of nat- * Mrs. CLINTON. Mr. President, I am s? Every proud to congratulate the Honorable orehouse. Harriet Cornell on her historic selecy derived tion as chair of the Rockland County ities. All erved for Legislature for a fifth consecutive eeds. No year. Harriet is the first chair of the in direct legislature to hold the office for 5 conrhen it is secutive years, it is the Harriet Cornell has been a member of the only the Rockland County Legislature since ublic and 1984. In her first year of office. Mrs, icient fu- Cornell founded the Legislatures Coiwith any wtany mssion on Womens Issues and invited Roy community leaders to participate in the formulation of public policy. She is ents rely also the chair of the Eleanor Roosevelt can have Legacy Committee, he end of Her long record of accomplishments ny dying led the Journal News naming her as syptoms one of 25 people who made the greatest iving dis- impact on Rockland County during the r] people aging 50 20th century. As chairwoman. Mrs, is cost in Cornells priorities have included proat times tection of our environment, enhanced ng to the educational resources, improved health is much services for women and chidren, homeveness of land security. Rocklands transportation infrastructure, and smart land Boise, use planning. Under her leadership, she has brought together elected officials TS from every level of government in Summit meetings to collaborate on these issues. HN I commend Mrs. Cornell for her many years of devoted public service to the t, it is citizens of Rockland County., ect that one of MESSAGE FROM THE HOUSE ed citip John At 6:09 p.m., a message from the e third House of Representatives, delivered by oro. Mrs. Cole, one of its reading clerks, anver thenounced that the House agreed to the cese of following resolutions: 32 coun- H. Res, 1. Resolution that Lorraine C. Milhools, 2 ler of the State of Texas, be, and is hereby, chosen Clerk of the House of Representanentary tives. That Wilson S. Livingood of the Coiy in the monwealth of Virginia, be, and is hereby, chosen Sergeant at Arms of the House of diocese. Representatives. That Daniel P. Beard of the he more State of Maryland be, and is hereby, chosen ge num- Chief Administrative Officer of the House of igrants. Representatives, and That Father Daniel P, recruits Coughlin of the State of Illinois, be, and is adAhereby, chosen Chaplain of the House of Repand Afresentatives. McRaith H. Res, 2. Resolution notifying the Senate Diocese that a quorum of the House of Representaa to 1ev- tives has assembled, that NANCY PELOSI, a Representative from the State of California, has been elected Speaker, than Lorraine C. Miller, a citizen of the State of Texas, has been elected Clerk of the House of Representatives of the One Hundred Eleventh Congress. The message also announced that the House has agreed to the following concurrent resolution, in which it requests the concurrence of the Senate: H. Con. Res, 1. Concurrent resolution regarding consent to assemble outside the seat of government. The message further announced that the Speaker appoints as members of the committee on the part of the House to join a committee on the part of the Senate to notify the President of the United States that a quorum of each House has assembled, and Congress is ready to receiver any communication that he may be pleased to make: The gentleman from Maryland Mr. HOYER and the gentleman from Ohio Mr. BOEHNER.',
                   'Mr. President, as the 111th Congress begins, the most important item on our agenda is to help end the worst economic crisis America has faced since the Great Depression, I look forward to working with my colleagues in the Senate to develop and approve an economic turnaround package as quickly as possible. But even if Congress authorizes as much as $1 trillion in new Government spending over the next 2 years to stimulate the economy, if we dont address the origins of this crisis, I fear the impact of any recovery package will be dampened. This economic crisis began with the bubble that burst in the housing market. So we have to address that, first and foremost. Families need to be able to stay in their homes, and communities need to be stabilized before the economy can start to grow again. Thats why, as my first bill in the new Congress, I am reintroducing the Helping Families Save Their Homes in Bankruptcy Act. When I first began working on this bill almost two years ago, the Center for Responsible Lending. Credit Suisse, and others estimated that 2 million homes were at risk of foreclosure. The Mortgage Bankers Association and the rest of the mortgage industry scoffed at such a number. Last month. Credit Suisse estimated that 8.1 million homes are likely to be lost to foreclosure by 2012. If the economy continues to worsen, they believe foreclosures will exceed 10 million homes. If over 8 million familiesrepresenting 16 percent of all mortgagesare losing their homes, our economy is not going to recover, I first introduced this bill in September of 2007, I have chaired three hearings on the subject and tried three times to pass this legislation last year. A large coalition supports this billincluding the AARP, the Consumer Federation of America, the Leadership Conference on Civil Rights, the AFLCIO, the Center for Responsible Lending, the National Association of Consumer Bankruptcy Attorneys, and many others. But the Mortgage Bankers Association and the rest of the mortgage industry have successfully opposed it so far. Three things have fundamentally changed, and I am back, pressing even harder that we make this bill law. First, the banks that brought us the reckless lending, dense securitization, and risky investing practices that created the boom and bust in the housing market have now happily accepted a $700 billion handout from the American taxpayers . . , even as most of them refuse to help the homeowners who are suffering most acutely from their irresponsible business practices. Frankly, I think that the credibility of the opposition to my bill has slipped just a bit. Second, it is painfully clear that foreclosure mitigation efforts to date have failed. Professor Alan White of the Valparaiso School of Law analyzed a large sample of the mortgage modifications made voluntarily by the industryled Hope Now Alliance. He found that almost half of these socalled foreclosure prevention plans actually increased the monthly payments of homeowners. How does that help families save their homes? Third. America soon will have a President who understands the enormity of this problem and supports this change to the bankruptcy code. So what does this bill do? This bill would allow mortgages on primary residences to be modified in bankruptcy just like other debtsincluding vacation homes, family farms, and yachts. Only families living in the home would qualifyno speculators are allowed. The bill would allow judges to cut through all of the constraints that have doomed foreclosure prevention plans from being successful for even the most proactive and wellintentioned mortgage servicers. There are very real constraints on some of the current efforts to prevent foreclosure today because most mortgages are sliced and sold to different investors, servicers sometimes have a hard time locating all of the owners of the mortgages to get their consent for modifications. Servicers that modify mortgages without the consent of all the investors fear that they could be sued. Some investors refuse to approve sensible restructurings, because there is little incentive for the owner of a second mortgage to approve a modification of a first mortgage that will see the second mortgage wiped out. Mortgage modifications that ignore the other pile of debt a household is facing is a setup for failure. Thats a leading reason why we see so many redefaults on newly modified mortgages through the current programs. Finally, servicers who are on the front lines answering the phone calls from homeowners and processing the paperwork often are compensated more for foreclosures than modifications. My proposal would allow judges to cut through these complicating factors to rework the underlying loans. The mortgages that are modified in bankruptcy will provide far more value to the lenders and the investors than foreclosure. The bill would provide borrowers who are frustrated with their mortgage servicers some desperately needed leverage to get their bankers full attention. It provides an incentive for banks to modify loans before the judges in bankruptcy do it for them. Best of all, this program would cost the taxpayers nothing. Given the staggering amounts that taxpayers have been asked to give to the mortgage industry lately, the taxpayers are ready for a plan that doesnt cost them anything and that will actually work. Since the Mortgage Bankers Association still opposes this plan, after taking all of that taxpayer money and after failing to do anything meaningful on their own to address this crisis, I want to address their primary remaining objection to this plan as clearly as possible so that everyone listening fully understands why the industry is wrong, once and for all. A few weeks ago, the Chairman of the Mortgage Bankers Association testified in the Senate Judiciary Committee that my bill would create a tax of $295, per month, for every homeowner in America, forever, I asked in the hearing, and my staff asked three times after the hearing, for some shred of evidence to support such a ridiculous claim. The response finally came just before the holidays, and it is laughable. The Mortgage Bankers Association claims that changing the bankruptcy code will create new costs for lenders that must then be passed on to all borrowers. They have concocted a list of individual costs that add up to the full "tax." as they call it. But they dont provide a single shred of evidence to support any of these cost estimates. Not one. They just made them all up. On the other hand, a study conducted by Adam Levitin of the Georgetown Law School uses actual statistical data to show that there is virtually no impact on mortgage interest rates just because mortgages can be modified by judges in bankruptcy. The main problem with the argument that my bill will increase future mortgage rates is this: The choice for mortgage lenders and investors is not full payment of the original mortgage versus a lower payment from a judicially modified mortgage. The choice is between a lower payment from a judicially modified mortgage and mortgage failure. Valparaisos Professor White reports that in his large study sample, mortgage servicers and their investors lost an average of 5 percent of the value of the mortgages that failed through foreclosure, or about $145.000 per loan. If those loans would have been modified in bankruptcy, the servicers and investors would have been given ownership of a sustainable mortgage worth at least the fair market value of the home plus an interest rate that included a premium for risk. These modified mortgages would on average have created far better results than the foreclosures that actually occurred. Therefore, when the Mortgage Bankers Association claims with no evidence whatsoever that my bill would raise mortgage interest rates, we should all ask them this: Why would mortgage bankers charge future borrowers higher interest rates tomorrow because of a change in the law that helps the bankers reduce their losses today? I urge the Senate to move swiftly to enact the economic recovery package that America desperately needs. And as part of that effort I urge my colleagues to support the remedy to the foreclosure crisis that will provide the most help to the 8.1 million families across the country who are at risk of losing their homes. If we dont address the core of the crisis, I fear that the stimulus may not work as well as It should, I look forward to working with Chairman DODD. Senator SCHUMER, all of the other Senators who have supported this provision, and Presidentelect Obama to see that It is signed into law quickly. Mr. President, I ask unanimous consent that the text of the bill be printed in the RECORD.',
                   ' Mr. President, I am offering today private relief legislation to provide lawful permanent residence status to Claudia Marquez Rico, a Mexican national living in Redwood City. CA. Born in Jalisco. Mexico. Claudia was brought to the United States by her parents 16 years ago. Claudia was just 6 years old at the time. She has two younger brothers. Jose and Omar, who came to America with her, and a sister. Maribel, who was born in California and is a U.S. Citizen. America is the only home they know. Eight years ago that home was visited by tragedy. As Mr, and Mrs. Marquez were driving to work early on the morning of October 4, 2000, they were both killed in a horrible traffic accident when their car collided with a truck on an isolated rural road. The children went to live with their aunt and uncle. Hortencia and Patricio Alcala. The Alcalas are a generous and loving couple. They are U.S, citizens with two children of their own and took the Marquez children in and did all they could to comfort them in their grief. They supervised their schooling, and made sure they received the counseling they needed, too. The family is active In their parish at Buen Pastor Catholic Church, and Patricio Alcala serves as a youth soccer coach. In 2001, the Alcalas were appointed the legal guardians of the Marquez children. Sadly, the Marquez family received poor legal representation. At the time of their parents death. Claudia and Jose were minors, and qualified for special immigrant juvenile status. This category was enacted by Congress to protect children like them from the hardship that would result from deportation under such extraordinary circumstances, when a State court deems them to be dependents due to abuse, abandonment or neglect. Today, their younger brother Omar is a U.S. Citizen, due to his adjustment as a special immigrant juvenile. Unfortunately, the familys previous lawyer failed to secure this relief for Claudia, and she has now reached the age of majority without having resolved her immigration status, 55. Pt, 1 139 I should note that their former lawyer. Walter Pineda, is currently answering charges on 29 counts of professional incompetence and 5 counts of moral turpitude for mishandling immigration cases and appears on his way to being disbarred, I am offering legislation on Claudias behalf because I believe that, without it, this family would endure an immense and unfair hardship. Indeed, without this legislation, this family will not remain a family for much longer. Despite the adversity they encountered. Claudia finished school. She supports herself, her 17yearold sister. Maribel, and her younger brother Omar. Again, both Maribel and Omar are now U.S. Citizens. Claudia has no close relatives in Mexico. She has never visited Mexico, and she was so young when she was brought to America that she has no memories of it. How can we expect her to start a new life there now? It would be a grave injustice to add to this familys misfortune by tearing these siblings apart. This is a close family, and they have come to rely on each other heavily in the absence of their deceased parents. This bill will prevent the added tragedy of another wrenching separation. Given these extraordinary and unique facts, I ask my colleagues to support this private relief bill on behalf of Claudia Rico. Mr. President, I ask unanimous consent that the text of the bill be printed in the RECORD.',
                   'I thank my colleague from Tyler and Longview and Marshall and Henderson and all points east of Fort Worth, I appreciate his hosting this hour today. As we talk about Presidentelect Obamas stimulus package, I am very mindful that he currently has something north of a 65 to 70 percent approval rating. So you really dont want to pick a fight right off the bat with a fella whos in that high regard across the United States. But so , think as a minimum, we ought to give him a chance to begin to put some meat on the bone of all of these great ideas that have been kind of at thenot even the 10.000foot level but at the 50.000foot level and looking forward to the actual legislative language as to how some of this stuff is going to work, I applaud him for calling for no earmarks and for transparency and accountability. Thats exactly what we want to do. Im particularly encouraged that Vice President BIDEN has committed to oversee the spending of every single dollar personally. Given the growing size of this bill, he is going to be one very busy Vice President as be puts his green eyeshade on, his garters, and pulls his sleeves up, gets out his pen, and actually watches the writing of each one of those checks as he committed to doing the other day, I am a bit discouraged, though, that the overall process that was announced yesterday that he believedour new President believes that he can spend, or we can collectively spend our way out of this current economic recession, depressionwhatever you want to call it, whatever title you want to give itIm concerned that thats not an accurate way to do this. One way to look at this would be to say, all right. If government spending is a panacea for the economy, if it will build a great economy, then looking at the spending, the government spending for the last 2 yearswhich I believe this Federal Government has spent more money in the last 2 years than any other 2year period in historythat certainly didnt drive a wonderful economy. Were in a bad economy right now. So if the premise is government spending builds economies, then we ought to be in a good economy right now. Quite frankly, we arent in that economy. The centerpiece, as both of our colleagues have talked about, is job creation. And at the end of the day, it really should be about jobs, I participated in a needs assessment in Midland County back in the United Way days. It was a zillion years ago. It was a process where you went through and asked people what was going on in their homes, what was going on in the neighborhoods, in local communities, what were the problems, what were the issues. We culled that down through some science to the top 10 needs for the Midland community. If you looked at those 10, nine of those 10 would have been favorably addressed by a job, by somebody having a job. And so it isin an arena where hyperbole is the norm, its difficult to overstate how important jobs are to an economy. And thats just the foundation, the base of those, I would also argue, though, that government jobsand my colleague and I from Texas have two really good government jobs. These government jobs that we have, we make money at it, and they are here forever. And some government jobs will always be here forever. But the jobs that would be created with the program thats been, you know, kind of highlighted at the 50.000foot level, those jobs shouldnt be forever. And when you dont talk about forever with a job, then that job is, by nature, temporary, and since its temporary, its hard for families to make plans based on a temporary job. Its hard for communities to plan on thosethe impact that those jobs have. So that temporariness of those government jobs lends itself to continued uncertainty, to continued anxiety about what happens when this ends, what happens when this is over as opposed to a business that comes into or locates into a community, begins to put down roots and build jobs and build wealth, add to the local tax rolls. All of the kinds of things the private sector jobs do, those have a sense of permanency to them that is just right. That makes sense to us. And I would argue that whatever we do on a goforward basis, that we focus more on private sector jobs and do whatever we can to avoid creating government jobs because once you put people on the government payroll, its hard to get them off and it does not build wealth, I would also like to point out that while our current circumstances are dire and difficult and hard and there is a lot of pain in the country right now, it is temporary. As weve seen, expanding economies are temporary. Weve enjoyed about a 7- or 8year good run with the expanding economy. Everybody enjoys that. New jobs are created, new wealth is created, opportunities. Everybody likes that. But those are temporary as weve now seen with this contracting economy. Well, the converse is true as well. Contracting economies are temporary. They may last a lot longer than wed like, a lot longer than wed enjoy, but at the end of the day, this world economy, this U.S, economy will turn the corner and will begin to expand. So as we look at what we do to address this issue, lets be careful that we dont take money to be earned by future generations to fix a temporary issue that were dealing with, I would argue that my colleagues and my generation, the last 4 years we have elevated this idea of taking somebody elses moneyin most instances its our grandkids and great grandkids and greatgreat grandchildrens moneyand lets fix todays problems. Which means that we have robbed our future generations of the money that theyre going to earn that they should have available to them to address their problems. Because they will have problems. There is nothing we can do today thats going to fix everything permanently, and those future generations have a right to the money they earn by the sweat of their brow. The problem is you and I are spending it. Collectively. Theres plenty of blame to go around. This isnt a partisan issue. Democrats. Republicans bear equal blame in this regard that weve constantly become addicted, in effect, to using borrowed money to address issues. And the issue were going to address over the next several weeks is this economy, and everything Ive heard so far is that were going to use borrowed money, I was in Fredericksburg. Texas, back in October doing a town hall meeting at an elementary school. If my colleagues have never done a town hall meeting in an elementary school, I would encourage it because you get some of the best questions ever from fifth graders, I was doing my best Q&A kind of thing, and this little fella In the second row raised his hand, and I recognized him, and he said. Mr. Congressman, what is the plan to pay off the national debt? And I said. Excuse me? He said. Yes, sir. Whats the plan to pay off the national debt? And I said. Young man, that is the single best question I have been asked while Ive been in Congress. There is no plan to pay off the national debt. Every dollar that we borrow is, in effect, permanently borrowed forever. Lets just take an example. Im a CPA so some of this comes a little bit easy to me. Weve got $11 trillion in hard debt. Debt weve got paper on, not counting the promises of Social Security. Medicare, and all of that. Weve got $11 trillion. In order to pay that off, this government has to run an $11 trillion surplus counting the interest. Its more than that if youve got interest. Given the history of the last 42 years, weve, I think, run surpluses 3 of those years. Thirtynine of them or fortyone of them, whatever the number is, have been deficits. So if anyone in their right mind thinks this Federal Government, given the propensity we have for spending other peoples money, can run a $12 or $11 trillion surplus in order to pay off the national debt, they are delusional beyond all words. Now, at a minimum, the first thing that we ought to do is quit doing whats gotten us to this point. Quit spending money we dont have. You know, itsacross the aisle weve got two seemingly desperate ways of doing things. On our side we want to cut taxes, and the other side spends money but doesnt raise taxes. It ought to be this way: If youre going to spend the money, then have the political backbone to raise the taxes, or if youre not going to raise the taxes, have the political backbone to not spend the money. Well, weve had it on our side where we spent the money and borrowed it, and the other side wants to spend the money and raise taxes. And all weve done is spend money that we dont have. Its not ours. No family gets to do that, no small business gets to do that, no other government entity Im aware of, other than the Federal Government, gets to do that. My preference, if were going to have some sort of a stimulus work, would be to focus on tax policy, the money thats earned by good citizens, and that we, at the point of a badge, take away from them. That tax policy ought to be stable, it ought to be predictable, it ought to be put in place. It allows them to keep more of their money and create those private sector jobs. Lets take the example of businesses. Section 179 allows businesses to deduct immediately in the year of purchase a certain amount of money that they spend on equipment that they use in their business. By being able to deduct that, the taxes they would otherwise have paid on that amount of money, they can recycle into their business by hiring new people, investing in new product, investing in new capacity. All those kinds of things. So that, in my view, is a much more appropriate stimulus of the economy than to collect a bunch of money here in Washington D.C, and then begin to try to parcel it out across some of the projects that our colleague from New Jersey was talking about earlier in terms of how that money is going to be spent under the, quoteunquote, stimulus package and the conference of mayors, you know. The shopping list that theyve gone through is, in my view, a much better way to try to stimulate this economy. Truth be told, at the end of the day, the Federal Government has precious little to do with whether or not the economy expands or contracts. Thats driven by the decisions of millions of Americans to decide whether or not theyre going to buy something new, whether or not theyre making enough money to be able to afford that, whether or not their businessprospects for their business is good enough that theyll go to the bank and borrow money and continue to begin to turn this corner. Those decisions are made all over the United States, all over the world by good, honest folks and not governments. So we sometimes delude ourselves into thinking thatand most of us are of the kind of personality that we came here to fix stuff, we came here to make this country a better place, we came here to do all those kinds of things. Sometimes its not our job. Our propensity is that we want to fix stuff, we want to do things to help this country. And when we see a problem as staggering and difficult as this one, we think that theres something we in fact can do, and we feel almost inadequate when we propose not doing something. But maybe in this instance, letting us absorb the pain and understand that in a deleveraging circumstance, when youre paying off debt as we are right now, that that does not grow an economy, but that does lay the foundation for that future economy that will begin to expand that we will all enjoy on a go forward basis. So if anybody remembers one thing Ive said today, it is, lets begin to look and lay a foundation for stopping fixing temporary problems with permanent debt that were borrowing from future generations and are hamstringing them and are hobbling their ability to take care of their issues when they are grown and in our position. So I appreciate my colleague for hosting this hour today.']
preds = model.predict(
    ["question-generation: " + description for description in american_speeches]
)

In [9]:
for speech in range(len(preds)):
  print('Context')
  print(american_speeches[speech])
  print('Generated')
  for generated in preds[speech]:
    print(generated)
  print()

Context
Mr. Speaker, I thank the gentleman. Mr. Speaker, you dont have to look far to see that families across this country are gripped with a tremendous amount of fear and uncertainty. They fear for their jobs, if they have one. They fear for their future as they see their 401(k)s, their college savings accounts collapse. They fear that their elected leaders dont get it. They fear that this Congress may very well be incapable of change, incapable of producing the kind of results that they want and to get it right. Under existing House rules, when a bill is brought to the floor that includes a tax increase, the minority has a right to offer a motion to strike that increase, and the Republican minority had done that on nearly half a dozen occasions over the past 2 years. With this rule change now, though. House Democrats are trying to push through what we Republicans will no longer have, the ability to say "no" to higher taxes. We will not be able to simply strike a tax increase and dem